In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pickle

In [9]:
# Ruta al directorio de las imágenes
image_directory = 'dataset'

In [10]:
# Dimensiones de las imágenes de entrada
img_width, img_height = 150, 150

In [11]:
# Hiperparámetros del modelo
batch_size = 32
epochs = 5

In [12]:
# Preprocesamiento y aumento de datos
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [13]:
# Cargar y dividir los datos de entrenamiento y validación
train_data = datagen.flow_from_directory(
    image_directory,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_data = datagen.flow_from_directory(
    image_directory,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 9600 images belonging to 12 classes.
Found 2400 images belonging to 12 classes.


In [33]:
# Construir el modelo de la red neuronal convolucional
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [35]:
# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [36]:
# Entrenar el modelo
model.fit(train_data, epochs=epochs, validation_data=validation_data)

Epoch 1/5
300/300 [==============================] - 82s 272ms/step - loss: 0.2889 - accuracy: 0.9155 - val_loss: 0.2869 - val_accuracy: 0.9167
Epoch 2/5
300/300 [==============================] - 86s 287ms/step - loss: 0.2869 - accuracy: 0.9167 - val_loss: 0.2868 - val_accuracy: 0.9167
Epoch 3/5
300/300 [==============================] - 80s 266ms/step - loss: 0.2868 - accuracy: 0.9167 - val_loss: 0.2868 - val_accuracy: 0.9167
Epoch 4/5
300/300 [==============================] - 81s 272ms/step - loss: 0.2868 - accuracy: 0.9167 - val_loss: 0.2869 - val_accuracy: 0.9167
Epoch 5/5
300/300 [==============================] - 79s 264ms/step - loss: 0.2868 - accuracy: 0.9167 - val_loss: 0.2869 - val_accuracy: 0.9167


In [37]:
filename = "./ml/model.pkl"
pickle.dump(model, open(filename, "wb"))

In [43]:
import mlflow

def create_exp_and_register_model(experiment_name, run_name,model = None,):

    mlflow.set_tracking_uri("http://localhost:5000")
    mlflow.set_experiment(experiment_name)

    with mlflow.start_run(run_name=run_name) as run:
        mlflow.set_tag("tag1", "NN")
        mlflow.sklearn.log_model(model, "model", registered_model_name='fruit-classifier')

experiment_name = "clasificador_basico"
run_name = "initial-run"

create_exp_and_register_model(experiment_name, # Nombre del experimento
                  run_name, # Nombre de la ejecución
                  model, # Modelo ajustado
                )

2023/07/09 21:51:04 INFO mlflow.tracking.fluent: Experiment with name 'clasificador_basico' does not exist. Creating a new experiment.
2023/07/09 21:51:23 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/h2/1lt2yrls5x3gx54fkln4wm3c0000gn/T/tmpd07u2s9c/model/model.pkl, flavor: sklearn), fall back to return ['scikit-learn==1.3.0', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback.
/Users/kevin.alvarez/personal/ws/m16-mlops/.conda/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'fruit-classifier'.
2023/07/09 21:51:23 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: fruit-classifier, version 1
Created version '1' of model 'fruit-classifier'.


In [45]:
client = mlflow.tracking.MlflowClient()
client.transition_model_version_stage(
    name="fruit-classifier",
    version=1,
    stage="Production"
)

<ModelVersion: aliases=[], creation_timestamp=1688953883466, current_stage='Production', description='', last_updated_timestamp=1688953942574, name='fruit-classifier', run_id='79c1ca9e672a47c8a06a40892c1b116a', run_link='', source='mlflow-artifacts:/325871729045372542/79c1ca9e672a47c8a06a40892c1b116a/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>